In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sn
sn.set()
from sklearn.linear_model import LinearRegression

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
raw = pd.read_csv('RL_slr.csv')
data = pd.DataFrame(raw)
data.head()

Preprocessing

In [ ]:
data.describe(include ='all')
## include all will do for cato data too

In [ ]:
data = raw.drop(['Model'],axis=1)

In [ ]:
data.isnull().sum()

In [ ]:
no_MV = data.dropna(axis=0)

In [ ]:
no_MV.describe()

PDFs

In [ ]:
sn.distplot(no_MV['Price'])

In [ ]:
### Dealing with outliers by removing top 10%
q = no_MV['Price'].quantile(0.99)
data_1 = no_MV[no_MV['Price']<q]
data_1.describe(include='all')

In [ ]:
sn.distplot(data_1['Price'])

In [ ]:
sn.distplot(data_1['Mileage'])

In [ ]:
q= data_1['Mileage'].quantile(0.99) ## select data of 99 quantile of data..as 1 % is outlier
data_2 =data_1[data_1['Mileage']<q]


In [ ]:
sn.distplot(data_2['Mileage'])

In [ ]:
sn.distplot(no_MV['EngineV'])

In [ ]:
data_3 =data_2[data_2['EngineV']<6.5]

In [ ]:
sn.distplot(data_3['EngineV'])

In [ ]:
sn.distplot(no_MV['Year'])

In [ ]:
q =data_3['Year'].quantile(0.01)
data_4=data_3[data_3['Year']>q]

In [ ]:
sn.distplot(data_4['Year'])

In [ ]:
cleaned_data= data_4.reset_index(drop=True) ### drop to ignore all the old indexes

In [ ]:
cleaned_data.describe()

OLS Assumpations

In [ ]:
f, (ax1,ax2,ax3) = plt.subplots(1, 3,sharey=True,figsize=(15,3))
ax1.scatter(cleaned_data['Year'],cleaned_data['Price'])
ax1.set_title('Price and Year')
ax2.scatter(cleaned_data['EngineV'],cleaned_data['Price'])
ax2.set_title('Price and EingineV')
ax3.scatter(cleaned_data['Mileage'],cleaned_data['Price'])
ax3.set_title('Price and Milleage')
plt.show()

The above patterns are not linaer so we have to transfornm

In [ ]:
sn.distplot(cleaned_data['Price'])

## you can see Price daat is not linaer which affects others also

Relaxing the assumputions

In [ ]:
## Use log transformation
log_price = np.log(cleaned_data['Price'])
cleaned_data['Log_Price'] = log_price
cleaned_data

In [ ]:
f, (ax1,ax2,ax3) = plt.subplots(1, 3,sharey=True,figsize=(15,3))
ax1.scatter(cleaned_data['Year'],cleaned_data['Log_Price'])
ax1.set_title('Price and Year')
ax2.scatter(cleaned_data['EngineV'],cleaned_data['Log_Price'])
ax2.set_title('Price and EingineV')
ax3.scatter(cleaned_data['Mileage'],cleaned_data['Log_Price'])
ax3.set_title('Price and Milleage')
plt.show()

In [ ]:
cleaned_data= cleaned_data.drop(['Price'], axis=1)

In [ ]:
cleaned_data

In [ ]:
## Check for Multicolinearity
cleaned_data.columns.values

In [ ]:
## SK learn can be used to check it
from statsmodels.stats.outliers_influence import variance_inflation_factor
vars = cleaned_data[['Mileage','EngineV','Year']]
vif = pd.DataFrame()
vif['VIF'] = [variance_inflation_factor(vars.values,i) for i in range (vars.shape[1])]
vif['Features'] = vars.columns
# a VIF of 1 means no multi_colinearity
## between 1 and 5 is good
vif

In [ ]:
no_MC = cleaned_data.drop(['Year'],axis =True)

Dummies

The number of dummies should be n-1 dummies becuase  the if all are 0 it menas it is the one we left behind



In [ ]:
dum_data = pd.get_dummies(no_MC,drop_first=True) ## drop first is n-1
dum_data

Rearrange a bit

In [ ]:
dum_data.columns.values

In [ ]:
cols = ['Log_Price', 'Mileage', 'EngineV', 'Brand_BMW',
       'Brand_Mercedes-Benz', 'Brand_Mitsubishi', 'Brand_Renault',
       'Brand_Toyota', 'Brand_Volkswagen', 'Body_hatch', 'Body_other',
       'Body_sedan', 'Body_vagon', 'Body_van', 'Engine Type_Gas',
       'Engine Type_Other', 'Engine Type_Petrol', 'Registration_yes']

In [ ]:
ready_data = dum_data[cols]
ready_data.head()

Regression

In [ ]:
target = ready_data['Log_Price']
inputs = ready_data.drop(['Log_Price'],axis=1)

Scale The Data

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(inputs)


In [ ]:
scaled_inputs = scaler.transform(inputs)

Traing and Testing Split

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test= train_test_split(scaled_inputs, target,test_size=0.2,random_state=42)

Regression Model

In [ ]:
REG = LinearRegression()
REG.fit(x_train,y_train)

In [ ]:
y_hat = REG.predict(x_train)

In [ ]:
plt.scatter(y_train, y_hat)
plt.xlabel('Targets (Y Train)',size=19)
plt.ylabel('Predictions (Y_Hat)',size=19)
plt.xlim(6,13)
plt.ylim(6,13)
plt.show()

In [ ]:
sn.distplot(y_train - y_hat)
plt.title('Residual Plot',size=20)

In [ ]:
REG.score(x_train,y_train)

Weights and Bias

In [ ]:
REG.intercept_

In [ ]:
REG.coef_

In [ ]:
tab = pd.DataFrame(inputs.columns.values, columns = ['Features'])
tab['Weights'] = REG.coef_
tab

#### Conntinuos Vars
## A positive weifght means the more the feature increases the more the target(Price) increases
## A negative ######################################################################## decreases

#### Dummy Vars
## A positive weight shows that the respective catogary is more expensive than the benchmark(Audi)
## A Negative weight shows that the respective catogary is less expensive than the benchmark(Audi)
##which was dropped by n-1 rule


###Never Compare dummies with contous vars.. dummies are for benchmark

Testing

In [ ]:
y_hat_test = REG.predict(x_test)

In [ ]:
plt.scatter(y_test,y_hat_test,alpha=0.2)
plt.xlabel('Targets (Y Tet)',size = 20)
plt.ylabel('Predictions (Y Test)',size = 20)
plt.xlim(6,13)
plt.ylim(6,13)
plt.show()

In [ ]:
df_pf = pd.DataFrame(np.exp(y_hat_test),columns=['Predictions'])
df_pf.head()

In [ ]:
df_pf['Target'] = np.exp(y_test)
df_pf.head()

In [ ]:
y_test = y_test.reset_index(drop=True)

In [ ]:
df_pf['Target'] = np.exp(y_test)
df_pf.head()

In [ ]:
df_pf['Residual (Difference)'] = df_pf['Target'] - df_pf['Predictions']

In [ ]:
df_pf['Difference in %'] = np.absolute(df_pf['Residual (Difference)']/df_pf['Target']*100)

##np.absute to show absoulte number and not rounded

In [ ]:
df_pf

In [ ]:
df_pf.describe()

In [ ]:
pd.options.display.max_rows = 999
pd.set_option('display.float_format',lambda x : '%.2f' % x)
df_pf.sort_values(by=['Difference in %'])